In [ ]:
import numpy as np
from numpy.typing import NDArray
from scipy.optimize import fmin_l_bfgs_b
from prettytable import PrettyTable
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
X = np.loadtxt('X.txt')
y = np.loadtxt('y.txt')

# We randomly permute the data
permutation = np.random.permutation(X.shape[ 0 ])
X = X[ permutation, : ]
y = y[ permutation ]

n_train = 800
X_train = X[ 0 : n_train, : ]
X_test = X[ n_train :, : ]
y_train = y[ 0 : n_train ]
y_test = y[ n_train : ]

## Short Lab Functions

In [ ]:
def plot_data_internal(X, y):
    x_min, x_max = X[ :, 0 ].min() - .5, X[ :, 0 ].max() + .5
    y_min, y_max = X[ :, 1 ].min() - .5, X[ :, 1 ].max() + .5
    xx, yy = np.meshgrid(np.linspace(x_min, x_max, 100), np.linspace(y_min, y_max, 100))
    plt.figure()
    plt.xlim(xx.min(None), xx.max(None))
    plt.ylim(yy.min(None), yy.max(None))
    ax = plt.gca()
    ax.plot(X[y == 0, 0], X[y == 0, 1], 'ro', label = 'Class 1')
    ax.plot(X[y == 1, 0], X[y == 1, 1], 'bo', label = 'Class 2')
    plt.xlabel('X1')
    plt.ylabel('X2')
    plt.title('Plot data')
    plt.legend(loc = 'upper left', scatterpoints = 1, numpoints = 1)
    return xx, yy

def plot_data(X, y):
    xx, yy = plot_data_internal(X, y)
    plt.show()

def logistic(x): return 1.0 / (1.0 + np.exp(-x))

def predict(X_tilde, w): return logistic(np.dot(X_tilde, w))

def compute_average_ll(X_tilde, y, w):
    output_prob = predict(X_tilde, w)
    return np.mean(y * np.log(output_prob) + (1 - y) * np.log(1.0 - output_prob))

def get_x_tilde(X): return np.concatenate((np.ones((X.shape[ 0 ], 1 )), X), 1)


def fit_w(X_tilde_train, y_train, X_tilde_test, y_test, n_steps, alpha):
    # Setup table
    table = PrettyTable()
    table.title="train/test log likelihood"
    table.field_names=["# it.", "train loss", "test loss"]

    w = np.random.randn(X_tilde_train.shape[ 1 ])
    #print(f"Shape o w: {w.shape}")
    ll_train = np.zeros(n_steps)
    ll_test = np.zeros(n_steps)
    for i in range(n_steps):
        sigmoid_value = predict(X_tilde_train, w)
        grad_ll = (y_train - sigmoid_value).T @ X_tilde_train
        #print(f"Grad ll shape:{grad_ll.shape}")
        w = w + alpha * grad_ll # Gradient-based update rule for w.
        ll_train[ i ] = compute_average_ll(X_tilde_train, y_train, w)
        ll_test[ i ] = compute_average_ll(X_tilde_test, y_test, w)
        table.add_row([i, ll_train[i], ll_test[i]])
    print(table)
    return w, ll_train, ll_test

def plot_ll(ll):
    plt.figure()
    ax = plt.gca()
    plt.xlim(0, len(ll) + 2)
    plt.ylim(min(ll) - 0.1, max(ll) + 0.1)
    ax.plot(np.arange(1, len(ll) + 1), ll, 'r-')
    plt.xlabel('Steps')
    plt.ylabel('Average log-likelihood')
    plt.title('Plot Average Log-likelihood Curve')
    plt.show()

def plot_predictive_distribution(X, y, w, map_inputs = lambda x : x):
    xx, yy = plot_data_internal(X, y)
    ax = plt.gca()
    X_tilde = get_x_tilde(map_inputs(np.concatenate((xx.ravel().reshape((-1, 1)), yy.ravel().reshape((-1, 1))), 1)))
    Z = predict(X_tilde, w)
    Z = Z.reshape(xx.shape)
    cs2 = ax.contour(xx, yy, Z, cmap = 'RdBu', linewidths = 2)
    plt.clabel(cs2, fmt = '%2.1f', colors = 'k', fontsize = 14)
    plt.show()

def get_confusion_matrix(X,y,w,tau=0.5):
    X_tilde = get_x_tilde(X)
    pred_soft = predict(X_tilde,w)
    y_hat = (pred_soft>tau)
    TP = np.count_nonzero(y_hat[y==1])
    FN = y_hat[y==1].shape[0] - TP
    FP = np.count_nonzero(y_hat[y == 0])
    TN = y_hat[y == 0].shape[0] - FP

    return np.array(
        [[TN/(TN+FP),FP/(TN+FP)],
         [FN/(TP+FN),TP/(TP+FN)]]
    )

def evaluate_basis_functions(l, X, Z):
    X2 = np.sum(X**2, 1)
    Z2 = np.sum(Z**2, 1)
    ones_Z = np.ones(Z.shape[ 0 ])
    ones_X = np.ones(X.shape[ 0 ])
    r2 = np.outer(X2, ones_Z) - 2 * np.dot(X, Z.T) + np.outer(ones_X, Z2)
    return np.exp(-0.5 / l**2 * r2)

In [ ]:
plot_data(X, y)

In [ ]:
# We train the classifier
alpha = 0.001
n_steps = 40

X_tilde_train = get_x_tilde(X_train)
X_tilde_test = get_x_tilde(X_test)

w, ll_train, ll_test = fit_w(X_tilde_train, y_train, X_tilde_test, y_test, n_steps, alpha)

plot_ll(ll_train)
plot_ll(ll_test)

In [ ]:
plot_predictive_distribution(X, y, w)

In [ ]:
mtx = get_confusion_matrix(X_test,y_test,w)
print(f"Confussion matrix: {mtx}")

In [ ]:

# We expand the data

l = 0.01 # Width of the Gaussian basis function.

X_tilde_train = get_x_tilde(evaluate_basis_functions(l, X_train, X_train))
X_tilde_test = get_x_tilde(evaluate_basis_functions(l, X_test, X_train))

# We train the new classifier on the feature expanded inputs

alpha = 0.008 # Learning rate for gradient-based optimisation with basis functions.
n_steps = 5_000 # Number of steps of gradient-based optimisation with basis functions.

w, ll_train, ll_test = fit_w( X_tilde_train, y_train, X_tilde_test, y_test, n_steps, alpha)

# We plot the training and test log likelihoods

plot_ll(ll_train)
plot_ll(ll_test)

# We plot the predictive distribution

plot_predictive_distribution(X, y, w, lambda x : evaluate_basis_functions(l, x, X_train))

mtx = get_confusion_matrix(evaluate_basis_functions(l, X_test, X_train),y_test,w)
print(f"Confusion matrix: {mtx}")

## Optimization

In [ ]:
def get_hessian(
        w:NDArray,
        X_tilde:NDArray, # Use Phi here if using RBFs
        sigma_0:int = 1
):
    sigmoid_value = predict(X_tilde,w)
    v = sigmoid_value * (1- sigmoid_value) # Hadamard prod
    A =  X_tilde.T @ np.diag(v) @ X_tilde
    A = A + np.ones_like(A) / sigma_0 **2
    return A


def evaluate_objective(
        w:NDArray,
        X_tilde:NDArray,
        y:NDArray,
        sigma_0:float = 1.0
):
    log_sigmoid = np.log( predict(X_tilde,w) )
    return - (1/sigma_0**2) * np.dot(w,w) + np.dot(y, log_sigmoid) + np.dot( np.ones_like(y)-y, log_sigmoid)

def get_objective(
        X_tilde:NDArray,
        y:NDArray,
        sigma_0:float = 1.0
):
    return lambda w : evaluate_objective(w,X_tilde,y,sigma_0)

def evaluate_jacobian(
    w:NDArray,
    X_tilde: NDArray,
    y: NDArray,
    sigma_0: float = 1.0
):
    sigmoid_value = predict(X_tilde,w)
    return w / sigma_0 ** 2 - (y - sigmoid_value).T @ X_tilde

def get_jacobian(
        X_tilde: NDArray,
        y: NDArray,
        sigma_0: float = 1.0
):
    return lambda w : evaluate_jacobian(w,X_tilde,y,sigma_0)

def optimize(
        X_tilde: NDArray,
        y: NDArray,
        sigma_0: float = 1.0
): # L-BFGS-B algo to compute the MAP estimator for w  && Start optimization at origin
    J = get_objective(X_tilde,y,sigma_0)
    dJ = get_jacobian(X_tilde,y,sigma_0)
    w_hat, J_min, d = fmin_l_bfgs_b( func = J, x0 =np.zeros(X_tilde.shape[1]), fprime = dJ)
    table = PrettyTable()
    table.title="Optimization"
    table.field_names=["w_hat","J_min","warnflag", "grad", "nit"]
    table.add_row([w_hat,J_min,d["warnflag"], d["grad"], d["nit"]])
    print(table)
    return w_hat


## Laplace Approximation

## Test

In [ ]:
if __name__=="__main__":
    N= 30
    M= 2
    w = np.ones(1+M)
    X = np.ones([N,M])
    X_tilde =get_x_tilde(X)
    A = get_hessian(w,X_tilde)
    print(A)